In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
!pip install -q -U keras-tuner
import kerastuner as kt

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - ETA:  - ETA:  - 0s 0us/step


In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [5]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt') 

In [7]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [8]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 448 and the optimal learning rate for the optimizer
is 0.001.



In [9]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - ETA: 6:39 - loss: 2.4182 - accuracy: 0.18 - ETA: 16s - loss: 1.1096 - accuracy: 0.6250 - ETA: 9s - loss: 0.9439 - accuracy: 0.680 - ETA: 7s - loss: 0.8509 - accuracy: 0.71 - ETA: 6s - loss: 0.7915 - accuracy: 0.73 - ETA: 5s - loss: 0.7635 - accuracy: 0.73 - ETA: 4s - loss: 0.7490 - accuracy: 0.74 - ETA: 4s - loss: 0.7154 - accuracy: 0.75 - ETA: 4s - loss: 0.6931 - accuracy: 0.75 - ETA: 4s - loss: 0.6791 - accuracy: 0.76 - ETA: 3s - loss: 0.6614 - accuracy: 0.76 - ETA: 3s - loss: 0.6459 - accuracy: 0.77 - ETA: 3s - loss: 0.6365 - accuracy: 0.77 - ETA: 3s - loss: 0.6296 - accuracy: 0.77 - ETA: 3s - loss: 0.6191 - accuracy: 0.78 - ETA: 3s - loss: 0.6134 - accuracy: 0.78 - ETA: 3s - loss: 0.6100 - accuracy: 0.78 - ETA: 2s - loss: 0.6092 - accuracy: 0.78 - ETA: 2s - loss: 0.6000 - accuracy: 0.78 - ETA: 2s - loss: 0.5933 - accuracy: 0.79 - ETA: 2s - loss: 0.5874 - accuracy: 0.79 - ETA: